In [ ]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms

In [ ]:
# Connect to the DuckDB database, fetch data
con = duckdb.connect('my_database.duckdb')
df = con.execute("SELECT * FROM metric_mainnet_lvr_impact_analysis_all").fetchdf()
con.close()


# Group the dataframe by pool_name and category
grouped = df.groupby(['pool_name', 'category'])

# Create a plot for each pool and category combination
for (name, category), group in grouped:
    fig, ax = plt.subplots(figsize=(12, 4))
    
    # Create a line plot for each fee_type within the category
    for fee_type in group['fee_type'].unique():
        data = group[group['fee_type'] == fee_type]
        # Sort the data by avg_fee_tier to ensure the line is drawn correctly
        data = data.sort_values('avg_fee_tier')
        ax.plot(data['avg_fee_tier'] * 100, data['fees_change'], label=fee_type, marker='o')
    
    # Add the base_avg_fee_tier as a vertical line
    base_fee_tier = group['base_avg_fee_tier'].iloc[0]  # Assuming it's the same for all rows in a group
    if base_fee_tier <= 0.01:
        ax.axvline(x=base_fee_tier * 100, color='g', linestyle='--')
        
        # Add vertical label for the base fee tier line
        trans = transforms.blended_transform_factory(ax.transData, ax.transAxes)
        ax.text(base_fee_tier * 100, 0.5, f'Base Fee Tier: {base_fee_tier*100:.2f}%', 
                rotation=90, va='center', ha='left', color='g',
                transform=trans, transform_rotates_text=True)
    
    ax.set_title(f'Fees Change vs Average Fee Tier for {name}\nCategory: {category}')
    ax.set_xlabel('Average Fee Tier (%)')
    ax.set_ylabel('Fees Change (%)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Add a horizontal line at y=0 for reference
    ax.axhline(y=0, color='r', linestyle='--', alpha=0.5)
    
    # Adjust y-axis to show full range of data
    ax.set_ylim(group['fees_change'].min() - 5, group['fees_change'].max() + 5)
    
    # Set x-axis limit to 1% (which is 0.01 in decimal form)
    ax.set_xlim(0, 1)
    
    # Add text to indicate if base fee tier is out of view
    if base_fee_tier > 0.01:
        ax.text(0.99, 0.98, f'Base Fee Tier: {base_fee_tier*100:.4f}%', 
                transform=ax.transAxes, ha='right', va='top', 
                bbox=dict(facecolor='white', edgecolor='none', alpha=0.7))
    
    plt.tight_layout()
    plt.show()